### 도서 정보 추출

In [4]:
import tools.ProcessFiles as pre_b
excel = pre_b.get_excel_data(1) # i차 분석

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
import tools.ProcessFiles as pre_b
pre_b.move_jsons(1) # i차 분석

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

In [3]:
import tools.ProcessWorkbook.quizbook3 as quiz

# quiz.main(json_files[0])

## 전체 돌리기

In [3]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(3)
json_files

[]

In [2]:
excel = pf.get_excel_data(3)

In [3]:
# excel

In [6]:
# for j in json_files:
if True:
    j = '/Users/yejin/Desktop/Desktop_AICenter✨/SFAIcenter/data_yejin/FINAL/2C_0902/Lv2/SS0014_conversation/SS0014.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # new = {
    #     'file_id': str(excel.loc[id, 'ISBN']),
    #     'title': excel.loc[id, '도서명'],
    #     'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #     'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #     'cat3_specific': excel.loc[id, '비고'],
    #     'pub_date': str(excel.loc[id, '출판일'])[:10],
    #     'contents': [],
    # }

    # new['contents'] = pl2.fill_chapter(origin['contents'])
    
    
    # for i in range(len(origin['contents'])):
        # contents = origin['contents'][i]
        # c = extract_qna(contents)
        # page_contents = remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # c = contents
        # print(contents)  
    # new = pl2.fill_chapter(origin)
    new = pl2.merge_paragraphs(origin)

    with open(INPUT_PATH, "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)

In [5]:
id

<function id(obj, /)>

In [ ]:
# for js in json_files:
#     INPUT_PATH = js
#     BACKUP_PATH = INPUT_PATH + ".bak"

#     if not os.path.exists(INPUT_PATH):
#         raise FileNotFoundError(INPUT_PATH)

#     with open(INPUT_PATH, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     with open(BACKUP_PATH, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

#     contents: List[Dict[str, Any]] = data.get("contents", [])
#     data["contents"] = build_output(contents)

#     with open(INPUT_PATH, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
for i, j in enumerate(json_files):
    if '276709608' in j:
        print(i, j)

In [ ]:
import re
import json
from typing import List, Dict, Any

def process_footnotes(content: str, page: str) -> tuple[str, List[Dict[str, Any]]]:
    """
    본문에서 각주 표시(•, ••, •••)를 찾아서 태그로 변환하고, 
    add_info에 footnote 정보를 추가하는 함수
    
    Args:
        content: 페이지 내용
        page: 페이지 번호 (4자리)
    
    Returns:
        tuple: (변환된 본문, footnote add_info 리스트)
    """
    # 각주 패턴 찾기: •, ••, ••• 등
    footnote_pattern = r'([^•\n]*?)(•+)'
    
    # ───── 구분선 이후의 각주 내용들 추출
    footnote_section_pattern = r'─────\n(.*?)(?=\n[^•]|\Z)'
    footnote_section_match = re.search(footnote_section_pattern, content, re.DOTALL)
    
    footnotes = []
    footnote_counter = 1
    
    if footnote_section_match:
        footnote_section = footnote_section_match.group(1)
        
        # 각주 내용들을 개별적으로 추출
        footnote_items = re.findall(r'•+\n(.*?)(?=\n•|\Z)', footnote_section, re.DOTALL)
        
        # 본문에서 각주 표시를 찾아서 태그로 변환
        def replace_footnote(match):
            nonlocal footnote_counter
            text_before = match.group(1)
            dots = match.group(2)
            
            # 각주 번호 계산 (•의 개수)
            footnote_num = len(dots)
            
            # 해당하는 각주 내용 찾기
            if footnote_num <= len(footnote_items):
                footnote_content = footnote_items[footnote_num - 1].strip()
                
                # add_info에 추가할 footnote 정보
                footnote_info = {
                    "tag": f"note_{page}_{footnote_counter:04d}",
                    "type": "footnote",
                    "description": footnote_content,
                    "caption": None,
                    "file_path": None,
                    "bbox": None
                }
                footnotes.append(footnote_info)
                
                # 본문에서 각주 표시를 태그로 변환
                tag = f"{{note_{page}_{footnote_counter:04d}}}"
                footnote_counter += 1
                
                return text_before + tag
            else:
                return match.group(0)  # 매칭되는 각주 내용이 없으면 원본 유지
        
        # 본문에서 각주 표시를 태그로 변환
        modified_content = re.sub(footnote_pattern, replace_footnote, content)
        
        # ───── 구분선과 각주 내용 제거
        modified_content = re.sub(footnote_section_pattern, '', modified_content, flags=re.DOTALL)
        
    else:
        modified_content = content
    
    return modified_content, footnotes

# 테스트
test_content = """약세 전환의 가능성이 좀 더 크다고 생각해. 몇 가지 이유가 있는데, 그중에서 딱 한 가 지만 말해줄게. 바로, 외국인들이 이머징emerging• 주식을 매수하고 있다는 점이야. 외국인들은 달러의 중기적 강세를 전망할 때는 대체로 이머징 주식을 매수하지 않거든. 어떤 종목을 사서 5% 수익이 났는데, 달러가 10% 강해졌다면 오히려 5%의 손실이 나니까 말이야.
─────
•
새롭게 부상한다는 의미로 신흥 시장을 가리킨다.
이수
달러가 왜 약세를 보이기 시작했는지 알려주세요."""

modified, footnotes = process_footnotes(test_content, "0023")
print("변환된 본문:")
print(modified)
print("\n추출된 각주:")
for footnote in footnotes:
    print(json.dumps(footnote, ensure_ascii=False, indent=2))


In [ ]:
def process_json_footnotes(json_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    JSON 데이터의 모든 페이지에서 각주를 처리하는 함수
    
    Args:
        json_data: 원본 JSON 데이터
    
    Returns:
        처리된 JSON 데이터
    """
    processed_data = json_data.copy()
    
    if 'contents' in processed_data:
        for content_item in processed_data['contents']:
            if 'page_contents' in content_item and 'page' in content_item:
                page_num = content_item['page']
                original_content = content_item['page_contents']
                
                # 각주 처리
                modified_content, footnotes = process_footnotes(original_content, page_num)
                
                # 본문 업데이트
                content_item['page_contents'] = modified_content
                
                # add_info에 footnote 추가
                if 'add_info' not in content_item:
                    content_item['add_info'] = []
                
                # 기존 add_info에 새로운 footnote들 추가
                content_item['add_info'].extend(footnotes)
    
    return processed_data

# 실제 파일에 적용해보기
INPUT_PATH = '/Users/yejin/Desktop/Desktop_AICenter✨/SFAIcenter/data_yejin/FINAL/2C_0902/Lv2/SS0014_conversation/SS0014.json'

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    original_data = json.load(f)

# 각주 처리
processed_data = process_json_footnotes(original_data)

# 결과 확인 - 첫 번째 페이지만 출력
if processed_data['contents']:
    first_page = processed_data['contents'][0]
    print("처리된 첫 번째 페이지:")
    print(f"페이지: {first_page['page']}")
    print(f"본문 길이: {len(first_page['page_contents'])}")
    print(f"add_info 개수: {len(first_page.get('add_info', []))}")
    
    # add_info에 footnote가 있는지 확인
    footnotes_in_page = [item for item in first_page.get('add_info', []) if item.get('type') == 'footnote']
    print(f"추출된 각주 개수: {len(footnotes_in_page)}")
    
    if footnotes_in_page:
        print("\n첫 번째 각주:")
        print(json.dumps(footnotes_in_page[0], ensure_ascii=False, indent=2))


In [ ]:
def process_footnotes_improved(content: str, page: str) -> tuple[str, List[Dict[str, Any]]]:
    """
    개선된 각주 처리 함수
    """
    footnotes = []
    footnote_counter = 1
    
    # ───── 구분선 이후의 각주 섹션 찾기
    footnote_section_pattern = r'─────\n(.*?)(?=\n[^•\n]|\Z)'
    footnote_section_match = re.search(footnote_section_pattern, content, re.DOTALL)
    
    if not footnote_section_match:
        return content, footnotes
    
    footnote_section = footnote_section_match.group(1)
    
    # 각주 항목들을 추출 (•로 시작하는 각 줄)
    footnote_lines = []
    for line in footnote_section.split('\n'):
        line = line.strip()
        if line.startswith('•'):
            # • 제거하고 내용만 추출
            footnote_content = line[1:].strip()
            if footnote_content:
                footnote_lines.append(footnote_content)
    
    if not footnote_lines:
        return content, footnotes
    
    # 본문에서 각주 표시를 찾아서 태그로 변환
    # •, ••, ••• 패턴을 찾되, 단어 뒤에 오는 것만
    footnote_pattern = r'(\w+)(•+)'
    
    def replace_footnote(match):
        nonlocal footnote_counter
        word = match.group(1)
        dots = match.group(2)
        
        # 각주 번호 계산 (•의 개수)
        footnote_num = len(dots)
        
        # 해당하는 각주 내용 찾기
        if footnote_num <= len(footnote_lines):
            footnote_content = footnote_lines[footnote_num - 1]
            
            # add_info에 추가할 footnote 정보
            footnote_info = {
                "tag": f"note_{page}_{footnote_counter:04d}",
                "type": "footnote",
                "description": footnote_content,
                "caption": None,
                "file_path": None,
                "bbox": None
            }
            footnotes.append(footnote_info)
            
            # 본문에서 각주 표시를 태그로 변환
            tag = f"{{note_{page}_{footnote_counter:04d}}}"
            footnote_counter += 1
            
            return word + tag
        else:
            return match.group(0)  # 매칭되는 각주 내용이 없으면 원본 유지
    
    # 본문에서 각주 표시를 태그로 변환
    modified_content = re.sub(footnote_pattern, replace_footnote, content)
    
    # ───── 구분선과 각주 내용 제거
    modified_content = re.sub(footnote_section_pattern, '', modified_content, flags=re.DOTALL)
    
    return modified_content, footnotes

# 테스트
test_content = """약세 전환의 가능성이 좀 더 크다고 생각해. 몇 가지 이유가 있는데, 그중에서 딱 한 가 지만 말해줄게. 바로, 외국인들이 이머징emerging• 주식을 매수하고 있다는 점이야. 외국인들은 달러의 중기적 강세를 전망할 때는 대체로 이머징 주식을 매수하지 않거든. 어떤 종목을 사서 5% 수익이 났는데, 달러가 10% 강해졌다면 오히려 5%의 손실이 나니까 말이야.
─────
•
새롭게 부상한다는 의미로 신흥 시장을 가리킨다.
이수
달러가 왜 약세를 보이기 시작했는지 알려주세요."""

modified, footnotes = process_footnotes_improved(test_content, "0023")
print("변환된 본문:")
print(modified)
print("\n추출된 각주:")
for footnote in footnotes:
    print(json.dumps(footnote, ensure_ascii=False, indent=2))


In [ ]:
def process_all_footnotes(json_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    JSON 데이터의 모든 페이지에서 각주를 처리하는 완전한 함수
    """
    processed_data = json_data.copy()
    total_footnotes = 0
    
    if 'contents' in processed_data:
        for content_item in processed_data['contents']:
            if 'page_contents' in content_item and 'page' in content_item:
                page_num = content_item['page']
                original_content = content_item['page_contents']
                
                # 각주 처리
                modified_content, footnotes = process_footnotes_improved(original_content, page_num)
                
                # 본문 업데이트
                content_item['page_contents'] = modified_content
                
                # add_info에 footnote 추가
                if 'add_info' not in content_item:
                    content_item['add_info'] = []
                
                # 기존 add_info에 새로운 footnote들 추가
                content_item['add_info'].extend(footnotes)
                total_footnotes += len(footnotes)
                
                if footnotes:
                    print(f"페이지 {page_num}: {len(footnotes)}개 각주 처리됨")
    
    print(f"총 {total_footnotes}개 각주가 처리되었습니다.")
    return processed_data

# 실제 파일에 적용
print("각주 처리 시작...")
processed_data = process_all_footnotes(original_data)

# 결과 확인 - 각주가 있는 페이지들만 출력
print("\n=== 각주 처리 결과 ===")
for i, content_item in enumerate(processed_data['contents']):
    footnotes = [item for item in content_item.get('add_info', []) if item.get('type') == 'footnote']
    if footnotes:
        print(f"\n페이지 {content_item['page']} ({i+1}번째):")
        print(f"  - 각주 개수: {len(footnotes)}")
        for j, footnote in enumerate(footnotes[:2]):  # 처음 2개만 출력
            print(f"  - 각주 {j+1}: {footnote['description'][:50]}...")
        if len(footnotes) > 2:
            print(f"  - ... 외 {len(footnotes)-2}개")


In [ ]:
# 처리된 결과를 파일에 저장
OUTPUT_PATH = INPUT_PATH.replace('.json', '_footnotes_processed.json')

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

print(f"\n처리된 파일이 저장되었습니다: {OUTPUT_PATH}")

# 원본 파일도 백업 후 업데이트할지 선택
print("\n원본 파일을 업데이트하시겠습니까? (y/n)")
# 실제로는 사용자가 선택할 수 있도록 주석 처리
# choice = input()
# if choice.lower() == 'y':
#     with open(INPUT_PATH, "w", encoding="utf-8") as f:
#         json.dump(processed_data, f, ensure_ascii=False, indent=4)
#     print("원본 파일이 업데이트되었습니다.")
# else:
#     print("원본 파일은 그대로 유지됩니다.")


---
# add_info
{
    "tag": "q_0000_0001",
    "type": "question",
    "description": {
        "number": "",
        "question": "",
        "options": [],
        "options": null,
        "answer": "",
        "explanation": ""
    },
    "caption": [
        " 기출",
        "키워드: "
    ],
    "file_path": null,
    "bbox": null
},
{
    "tag": "note_0000_0001",
    "type": "footnote",
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.",
    "caption": null,
    "file_path": null,
    "bbox": null
},
{
    "tag": "tb_0000_0001",
    "type": "table",
    "description": "\\begin{tabular}{|c|c|c|c|}\n\\hline\n나라 이름 & 지수 이름 & 포괄 종목 & 작성 기관 \\\\\\hline\n\\multirow{2}{*}{한 국} & KOSPI & \\makecell[l]{유가증권시장 상장\\\\전종목} & 한국거래소 \\\\\\cline{2-4}\n& KOSDAQ & \\makecell[l]{코스닥 상장 전종목} & 한국거래소 \\\\\\hline\n\\multirow{4}{*}{미 국} & DJIA & \\makecell[l]{뉴욕거래소, 나스닥\\\\상장 30 개 종목} & 다우존스사 \\\\\\cline{2-4}\n& S\\&P 500 & \\makecell[l]{뉴욕거래소,\\\\미국거래소, 나스닥\\\\상장 500 개 종목} & \\makecell[c]{Standard \\&\\\\Poors 사} \\\\\\hline\n\\multirow{3}{*}{일 본} & TOPIX & \\makecell[l]{동경거래소 상장\\\\전종목} & 동경거래소 \\\\\\cline{2-4}\n& NIKKEI 225 & \\makecell[l]{동경거래소 상장\\\\225 개 종목} & 일본경제신문사 \\\\\\hline\n영 국 & FTSE 100 & \\makecell[l]{런던거래소 상장\\\\100 개 종목} & FTSE \\\\\\hline\n독 일 & DAX 30 & \\makecell[l]{프랑크푸르트 거래소\\\\상장 30 개 종목} & Deutsche Boerse \\\\\\hline\n프랑스 & CAC 40 & \\makecell[l]{Euronext Paris 상장\\\\40 개 종목} & Euronext \\\\\\hline\n중 국 & 상해종합지수 & \\makecell[l]{상해거래소 상장\\\\전종목} & 상해거래소 \\\\\\hline\n\\end{tabular}",
    "caption": [
        "표2. 주가지수 개요"
    ],
    "file_path": "??????/crop/tb_0000_0001.png",
    "bbox": null
},
{
    "tag": "img_0000_0001",
    "type": "image",
    "description": null,
    "caption": [
        "도표1: 경제주체간 상품, 생산요소 및 자금의 흐름"
    ],
    "file_path": "?????/crop/img_0000_0001.png",
    "bbox": null
}